In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import os
import keras
import tensorflow
import tensorflow.keras

In [2]:
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import logging.config
import math
import pkg_resources
import random

In [6]:
%pip install cfg_load


In [7]:
from mininet.net import Mininet
from mininet.net import Mininet,CLI
from mininet.node import OVSKernelSwitch, Host, RemoteController
from mininet.link import TCLink,Link
from mininet.log import setLogLevel, info

import time, os

import numpy as np


class MininetBackEnd(object):

    def init_params(self, mu, sigma, link_bw, sla_bw):
        self.mu = float(mu)
        self.sigma = float(sigma)
        self.sla_bw = float(sla_bw)
        self.link_lat = float(0.0)
        self.link_bw = float(link_bw)
        self.traffic_reset = 0
     
    def reset_links(self):
        self.current_link_failure = False
        self.previous_link_failure = False

        self.active_link = 0 # internet by default
        self.episode_over = False

        self.take_measurements()

    def __init__(self, mu, sigma, link_bw, sla_bw, seed):

        np.random.seed(seed)

        self.init_params(mu, sigma, link_bw, sla_bw)

        self.net = Mininet( topo=None, controller=RemoteController, ipBase='10.0.0.0/8')
        c0 = RemoteController( 'c0', ip='127.0.0.1', port=6653 )
        
        self.net.addController(c0)
        
        self.h1 = self.net.addHost('h1', mac = '00:00:00:00:00:01', ip='10.0.0.1')
        self.h2 = self.net.addHost('h2', mac = '00:00:00:00:00:02', ip='10.0.0.2')
        self.n1 = self.net.addHost('n1', mac = '00:00:00:00:00:03', ip='10.0.0.3')
        self.n2 = self.net.addHost('n2', mac = '00:00:00:00:00:04', ip='10.0.0.4')
        self.n3 = self.net.addHost('n3', mac = '00:00:00:00:00:05', ip='10.0.0.5')
        self.n4 = self.net.addHost('n4', mac = '00:00:00:00:00:06', ip='10.0.0.6')
        self.s1 = self.net.addSwitch('s1', cls=OVSKernelSwitch, protocols='OpenFlow13')
        self.s2 = self.net.addSwitch('s2', cls=OVSKernelSwitch, protocols='OpenFlow13')
        self.s3 = self.net.addSwitch('s3', cls=OVSKernelSwitch, protocols='OpenFlow13')
        self.s4 = self.net.addSwitch('s4', cls=OVSKernelSwitch, protocols='OpenFlow13')
        
        self.net.addLink( self.h1, self.s1, 0, 1, cls=Link)
        self.net.addLink( self.h2, self.s2, 0, 1, cls=Link)
        self.net.addLink( self.n1, self.s1, 0, 4, cls=Link)
        self.net.addLink( self.n2, self.s1, 0, 5, cls=Link)
        self.net.addLink( self.n3, self.s2, 0, 4, cls=Link)
        self.net.addLink( self.n4, self.s2, 0, 5, cls=Link)
        self.net.addLink( self.s1, self.s3, 2, 1, cls=TCLink, bw=self.link_bw)
        self.net.addLink( self.s1, self.s4, 3, 1, cls=TCLink, bw=self.link_bw)
        self.net.addLink( self.s2, self.s3, 2, 2, cls=TCLink, bw=self.link_bw)
        self.net.addLink( self.s2, self.s4, 3, 2, cls=TCLink, bw=self.link_bw)

        self.net.start()

        #GROUPS
        #s1
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-group s1 group_id=20,type=select,bucket=weight:5,watch_port:"s1-eth3",actions=output:"s1-eth3",bucket=weight:5,watch_port:"s1-eth2",actions=output:"s1-eth2"')
        #s2
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-group s2 group_id=10,type=select,bucket=weight:5,watch_port:"s2-eth3",actions=output:"s2-eth3",bucket=weight:5,watch_port:"s2-eth2",actions=output:"s2-eth2"')

        #FLOWS
        #s1
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.2,nw_dst=10.0.0.1,actions=output:"s1-eth1"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.1,actions=output:"s1-eth1"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.1,nw_dst=10.0.0.2,actions=group:20')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.2,actions=group:20')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.1,nw_dst=10.0.0.3,actions=output:"s1-eth4"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.3,actions=output:"s1-eth4"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.3,nw_dst=10.0.0.1,actions=output:"s1-eth1"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.1,actions=output:"s1-eth1"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.1,nw_dst=10.0.0.4,actions=output:"s1-eth5"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.4,actions=output:"s1-eth5"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.4,nw_dst=10.0.0.1,actions=output:"s1-eth3"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.1,actions=output:"s1-eth3"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.5,nw_dst=10.0.0.1,actions=output:"s1-eth1"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.1,actions=output:"s1-eth1"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.1,nw_dst=10.0.0.5,actions=group:20')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.5,actions=group:20')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.6,nw_dst=10.0.0.1,actions=output:"s1-eth1"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.1,actions=output:"s1-eth1"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.1,nw_dst=10.0.0.6,actions=group:20')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.6,actions=20')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.2,nw_dst=10.0.0.3,actions=output:"s1-eth4"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.3,actions=output:"s1-eth4"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.3,nw_dst=10.0.0.2,actions="s1-eth2"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.2,actions="s1-eth2"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.2,nw_dst=10.0.0.4,actions=output:"s1-eth5"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.4,actions=output:"s1-eth5"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.4,nw_dst=10.0.0.2,actions="s1-eth3"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.2,actions="s1-eth3"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.3,nw_dst=10.0.0.4,actions=output:"s1-eth5"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.4,actions=output:"s1-eth5"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.4,nw_dst=10.0.0.3,actions=output:"s1-eth4"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.3,actions=output:"s1-eth4"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.3,nw_dst=10.0.0.5,actions="s1-eth2"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.5,actions="s1-eth2"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.5,nw_dst=10.0.0.3,actions=output:"s1-eth4"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.3,actions=output:"s1-eth4"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.3,nw_dst=10.0.0.6,actions="s1-eth2"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.6,actions="s1-eth2"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.6,nw_dst=10.0.0.3,actions=output:"s1-eth4"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.3,actions=output:"s1-eth4"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.4,nw_dst=10.0.0.5,actions="s1-eth3"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.5,actions="s1-eth3"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.5,nw_dst=10.0.0.4,actions=output:"s1-eth5"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.4,actions=output:"s1-eth5"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.4,nw_dst=10.0.0.6,actions="s1-eth3"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.6,actions="s1-eth3"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 ip,nw_src=10.0.0.6,nw_dst=10.0.0.4,actions=output:"s1-eth5"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.4,actions=output:"s1-eth5"')
        self.s1.cmd('ovs-ofctl -O OpenFlow13 add-flow s1 priority=1,ipv6,actions=drop')
        #s2
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.2,nw_dst=10.0.0.1,actions=group:10')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.1,actions=group:10')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.1,nw_dst=10.0.0.2,actions=output:"s2-eth1"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.2,actions=output:"s2-eth1"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.5,nw_dst=10.0.0.1,actions="s2-eth2"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.1,actions="s2-eth2"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.1,nw_dst=10.0.0.5,actions=output:"s2-eth4"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.5,actions=output:"s2-eth4"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.6,nw_dst=10.0.0.1,actions="s2-eth3"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.1,actions="s2-eth3"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.1,nw_dst=10.0.0.6,actions=output:"s2-eth5"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.6,actions=output:"s2-eth5"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.2,nw_dst=10.0.0.3,actions=group:10')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.3,actions=group:10')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.3,nw_dst=10.0.0.2,actions=output:"s2-eth1"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.2,actions=output:"s2-eth1"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.2,nw_dst=10.0.0.4,actions=group:10')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.4,actions=group:10')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.4,nw_dst=10.0.0.2,actions=output:"s2-eth1"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.2,actions=output:"s2-eth1"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.2,nw_dst=10.0.0.5,actions=output:"s2-eth4"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.5,actions=output:"s2-eth4"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.5,nw_dst=10.0.0.2,actions=output:"s2-eth1"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.2,actions=output:"s2-eth1"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.2,nw_dst=10.0.0.6,actions=output:"s2-eth5"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.6,actions=output:"s2-eth5"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.6,nw_dst=10.0.0.2,actions=output:"s2-eth1"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.2,actions=output:"s2-eth1"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.3,nw_dst=10.0.0.5,actions=output:"s2-eth4"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.5,actions=output:"s2-eth4"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.5,nw_dst=10.0.0.3,actions="s2-eth2"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.3,actions="s2-eth2"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.3,nw_dst=10.0.0.6,actions=output:"s2-eth5"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.6,actions=output:"s2-eth5"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.6,nw_dst=10.0.0.3,actions="s2-eth3"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.3,actions="s2-eth3"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.4,nw_dst=10.0.0.5,actions=output:"s2-eth4"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.5,actions=output:"s2-eth4"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.5,nw_dst=10.0.0.4,actions="s2-eth2"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.4,actions="s2-eth2"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.4,nw_dst=10.0.0.6,actions=output:"s2-eth5"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.6,actions=output:"s2-eth5"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.6,nw_dst=10.0.0.4,actions="s2-eth3"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.4,actions="s2-eth3"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.5,nw_dst=10.0.0.6,actions=output:"s2-eth5"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.6,actions=output:"s2-eth5"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 ip,nw_src=10.0.0.6,nw_dst=10.0.0.5,actions=output:"s2-eth4"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.5,actions=output:"s2-eth4"')
        self.s2.cmd('ovs-ofctl -O OpenFlow13 add-flow s2 priority=1,ipv6,actions=drop')
        #s3
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.2,nw_dst=10.0.0.1,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.1,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.1,nw_dst=10.0.0.2,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.2,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.5,nw_dst=10.0.0.1,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.1,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.1,nw_dst=10.0.0.5,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.5,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.6,nw_dst=10.0.0.1,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.1,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.1,nw_dst=10.0.0.6,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.6,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.2,nw_dst=10.0.0.3,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.3,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.3,nw_dst=10.0.0.2,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.2,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.2,nw_dst=10.0.0.4,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.4,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.4,nw_dst=10.0.0.2,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.2,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.3,nw_dst=10.0.0.5,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.5,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.5,nw_dst=10.0.0.3,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.3,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.3,nw_dst=10.0.0.6,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.6,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.6,nw_dst=10.0.0.3,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.3,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.4,nw_dst=10.0.0.5,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.5,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.5,nw_dst=10.0.0.4,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.4,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.4,nw_dst=10.0.0.6,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.6,actions=output:"s3-eth2"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 ip,nw_src=10.0.0.6,nw_dst=10.0.0.4,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.4,actions=output:"s3-eth1"')
        self.s3.cmd('ovs-ofctl -O OpenFlow13 add-flow s3 priority=1,ipv6,actions=drop')
        #s4
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.2,nw_dst=10.0.0.1,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.1,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.1,nw_dst=10.0.0.2,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.2,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.5,nw_dst=10.0.0.1,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.1,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.1,nw_dst=10.0.0.5,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.5,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.6,nw_dst=10.0.0.1,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.1,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.1,nw_dst=10.0.0.6,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.1,arp_tpa=10.0.0.6,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.2,nw_dst=10.0.0.3,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.3,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.3,nw_dst=10.0.0.2,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.2,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.2,nw_dst=10.0.0.4,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.2,arp_tpa=10.0.0.4,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.4,nw_dst=10.0.0.2,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.2,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.3,nw_dst=10.0.0.5,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.5,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.5,nw_dst=10.0.0.3,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.3,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.3,nw_dst=10.0.0.6,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.3,arp_tpa=10.0.0.6,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.6,nw_dst=10.0.0.3,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.3,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.4,nw_dst=10.0.0.5,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.5,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.5,nw_dst=10.0.0.4,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.5,arp_tpa=10.0.0.4,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.4,nw_dst=10.0.0.6,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.4,arp_tpa=10.0.0.6,actions=output:"s4-eth2"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 ip,nw_src=10.0.0.6,nw_dst=10.0.0.4,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,arp,arp_spa=10.0.0.6,arp_tpa=10.0.0.4,actions=output:"s4-eth1"')
        self.s4.cmd('ovs-ofctl -O OpenFlow13 add-flow s4 priority=1,ipv6,actions=drop')

        #CLI(self.net)

        # start udp  traffic receiver - simulate other traffic
        #self.h4.cmd("iperf  -u -s -i 1 >& /tmp/udp_server.log &")

        # start host tcp traffic receiver - simulate main flow
        #self.h2.cmd("iperf  -s -i 1  >& /tmp/tcp_server.log &")
        
        #Start main traffic receiver
        self.h2.cmd('iperf -u -s &')
        
        #Start main traffic generator
        self.h1.cmd('iperf -u -c 10.0.0.2 -p 2 -l 128 -b 1K -t 60000000 >& /tmp/main_traffic.log &')
        #self.h1.cmd('ping -c2 10.0.0.2 >& /tmp/ping_result.log &')
        
        #Start udp traffic receivers
        self.n4.cmd('iperf -u -s &')
        self.n3.cmd('iperf -u -s &')

        self.reset_links()


    def cleanup(self):
        self.net.stop()
        
    def generate_traffic(self):
        print('generating new traffic')
        traffic1Bw = random.randint(0, 1)
        traffic2Bw = random.randint(0, 1)
        print('Traffic Link1: {0}, Traffic Link2: {1}'.format(traffic1Bw, traffic2Bw))
        self.n1.cmd('iperf -u -c 10.0.0.5 -b {0}K -l 128 -t 4 >& /tmp/traffic1.log &'.format(traffic1Bw))
        self.n2.cmd('iperf -u -c 10.0.0.6 -b {0}K -l 128 -t 4 >& /tmp/traffic2.log &'.format(traffic2Bw))
    
    
    
    def pull_from_json(self):
        with open("./numbers.json") as f:
            data = json.load(f)
            #print("switch 3 bw: ", data["switches_free_bw"]["3"]["1"])
            #print("switch 4 bw: ", data["switches_free_bw"]["4"]["1"])
            #print("switch 3 speed: ", data["switches_port_speeds"]["3"]["1"])
            #print("switch 4 speed: ", data["switches_port_speeds"]["4"]["1"])
            return data["switches_free_bw"]["3"]["1"], data["switches_free_bw"]["4"]["1"], data["switches_port_speeds"]["3"]["1"], data["switches_port_speeds"]["4"]["1"]


    def take_measurements(self):
        # send udp traffic  - simulate other flow
      
        #udp_bw1 = np.random.normal(self.mu, self.sigma) 
        #udp_bw2 = np.random.normal(self.mu, self.sigma) 
        time.sleep(1)
        self.current_bw1, self.current_bw2, self.speed1, self.speed2 = self.pull_from_json()
        # always measure internet link available bw
        #self.available_bw1 = float(self.link_bw) - float(udp_bw1)
        #self.available_bw2 = float(self.link_bw) - float(udp_bw2)

        #if self.available_bw1 < 0.0:
        #    self.available_bw1 = 0.0

            
        #if self.available_bw2 < 0.0:
        #    self.available_bw2 = 0.0
            
            
        #self.current_bw1 = np.random.normal(self.available_bw1, 3)
        #self.current_bw2 = np.random.normal(self.available_bw2, 3)
        print(self.current_bw1, self.current_bw2, self.speed1, self.speed2)
        if (self.traffic_reset == 0):
            self.generate_traffic()
        if (self.traffic_reset == 400):
            self.traffic_reset = 0
        else:
            self.traffic_reset += 1 

    def switch_flows(self, action):
        self.Link1Weight = 0
        self.Link2Weight = 0
        
        if action == 0:
            self.Link1Weight = 10
            self.Link2Weight = 0
        elif action == 1:
            self.Link1Weight = 9
            self.Link2Weight = 1
        elif action == 2:
            self.Link1Weight = 8
            self.Link2Weight = 2
        elif action == 3:
            self.Link1Weight = 7
            self.Link2Weight = 3
        elif action == 4:
            self.Link1Weight = 6
            self.Link2Weight = 4
        elif action == 5:
            self.Link1Weight = 5
            self.Link2Weight = 5
        elif action == 6:
            self.Link1Weight = 4
            self.Link2Weight = 6
        elif action == 7:
            self.Link1Weight = 3
            self.Link2Weight = 7
        elif action == 8:
            self.Link1Weight = 2
            self.Link2Weight = 8
        elif action == 9:
            self.Link1Weight = 1
            self.Link2Weight = 9
        elif action == 10:
            self.Link1Weight = 0
            self.Link2Weight = 10
        else:
            return
        
        cmd1 = "ovs-ofctl --protocols=OpenFlow13 mod-group  s2 group_id=10,type=select,bucket=weight:{0},watch_port:\"s2-eth3\",actions=output:\"s2-eth3\",bucket=weight:{1},watch_port:\"s2-eth2\",actions=output:\"s2-eth2\"".format(self.Link1Weight,self.Link2Weight)
        cmd2 = "ovs-ofctl --protocols=OpenFlow13 mod-group  s1 group_id=20,type=select,bucket=weight:{0},watch_port:\"s1-eth3\",actions=output:\"s1-eth3\",bucket=weight:{1},watch_port:\"s1-eth2\",actions=output:\"s1-eth2\"".format(self.Link1Weight,self.Link2Weight)
        #info(cmd)
        self.s1.cmd(cmd1)
        self.s2.cmd(cmd2)



    def switch_link(self, action):

        # if action specifies same link s before it is not a switch
        if action != self.active_link:
            self.switch_flows(action)
            

        ## action is 0 => link is internet
        ## action 1 => MPLS 
        self.active_link  = action

        self.take_measurements()

        ## Here is the logic that checks  two subsequent SLA failures
        self.current_link_failure = False

        # if current bandwidth less than SLA it is a failure
        if self.active_link == 0:
            if float(self.current_bw1) < float(self.sla_bw):
                info ('current link failure')
                self.current_link_failure = True

                # if it failed in previous tick also, mark it a link failure
                if  self.previous_link_failure == True:
                    info ('previous link also failure, episode over')
                    self.episode_over = True
            
        # copy current to previous
        self.previous_link_failure = self.current_link_failure 
        
        return self.episode_over 

        

In [8]:
import logging.config
import math
import pkg_resources
import random

# 3rd party modules
from gym import spaces
import cfg_load
import gym
import numpy as np




class SdwanEnv(gym.Env):
    """
    Define Sdwan environment.
    The environment defines  how links will be selected based on bandwidth
    availability 
    """

    def __init__(self, max_ticks=300):
        #self.__version__ = "0.1.0"
        #logging.info("SdwanEnv - Version {}".format(self.__version__))

        # General variables defining the environment

        self.LINK_BW = 5.0
        self.LINK_SELECT_ACTION_LEFT = 0
        self.LINK_SELECT_ACTION_LEFT_10 = 1
        self.LINK_SELECT_ACTION_LEFT_20 = 2
        self.LINK_SELECT_ACTION_LEFT_30 = 3
        self.LINK_SELECT_ACTION_LEFT_40 = 4
        self.LINK_SELECT_ACTION_LEFT_50 = 5
        self.LINK_SELECT_ACTION_LEFT_60 = 6
        self.LINK_SELECT_ACTION_LEFT_70 = 7
        self.LINK_SELECT_ACTION_LEFT_80 = 8
        self.LINK_SELECT_ACTION_LEFT_90 = 9
        self.LINK_SELECT_ACTION_RIGHT = 10
        self.MAX_TICKS = max_ticks

        self.backend = MininetBackEnd(mu=4, sigma=2, link_bw=self.LINK_BW, sla_bw=3, seed=100)

        # Define what the agent can do
        # Choose link1 or Link2 
        self.action_space = spaces.Discrete(11)

        # Observation 

        low = np.array([self.LINK_SELECT_ACTION_LEFT,  # active link
                        5.0,  #Path 1 bw
                        5.0,  #Path 2 bw
                        ])
        high = np.array([self.LINK_SELECT_ACTION_RIGHT, self.LINK_BW, self.LINK_BW])

        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        # episode over 
        self.episode_over = False
        self.info = {} 

        # Store what the agent tried
        self.curr_episode = -1
        self.action_episode_memory = []

    def step(self, action):
        """
        The agent takes a step in the environment.
        Parameters
        ----------
        action : int
        Returns
        -------
        ob, reward, episode_over, info : tuple
            ob (object) :
                an environment-specific object representing your observation of
                the environment.
            reward (float) :
                amount of reward achieved by the previous action. The scale
                varies between environments, but the goal is always to increase
                your total reward.
            episode_over (bool) :
                whether it's time to reset the environment again. Most (but not
                all) tasks are divided up into well-defined episodes, and done
                being True indicates the episode has terminated. (For example,
                perhaps the pole tipped too far, or you lost your last life.)
            info (dict) :
                 diagnostic information useful for debugging. It can sometimes
                 be useful for learning (for example, it might contain the raw
                 probabilities behind the environment's last state change).
                 However, official evaluations of your agent are not allowed to
                 use this for learning.
        """
        self.take_action(action)
        reward = self.get_reward()
        ob = self.get_state()
        return ob, reward, self.episode_over, self.info 

    def take_action(self, action):
        self.episode_over = self.backend.switch_link(action)
                
        self.ticks += 1

        # check if episode ended by ERROR, then mark it in 'info'
        if self.episode_over:
            logging.info ('Episode ended by ERROR')
            self.info['exit_status'] = 'ERROR'

        # else Stop if max ticks over
        elif self.ticks == self.MAX_TICKS:
            logging.info ('Max ticks over, ending episode')
            self.episode_over = True
            self.info['exit_status'] = 'NORMAL'

    def get_reward(self):

        print('current bw1:{0}, sla bw:{1}'.format(self.backend.current_bw1, self.backend.sla_bw))
        print('current bw2:{0}, sla bw:{1}'.format(self.backend.current_bw2, self.backend.sla_bw))
        
        print('Link1 Weight: {0}'.format(self.backend.Link1Weight))
        print('Link1 Weight: {0}'.format(self.backend.Link2Weight))

        # otherwise, reward for surviving this 'tick'
        reward = 1

        total_bw = self.backend.current_bw1 * ((self.backend.Link1Weight*10)/100) + self.backend.current_bw2 * ((self.backend.Link2Weight*10)/100)
        diff = total_bw - self.backend.sla_bw
        print('Total Bw: {0}, Wanted Bw: {1}, diff: {2}'.format(total_bw, self.backend.sla_bw, diff))
        
        reward += diff
        
        latency_fixed = 6
        #reward += self.backend.Link1Weight * latency_fixed + self.backend.Link2Weight * latency_fixed

        
        print('Chosen action:{0}, Reward:{1}'.format(self.backend.active_link, reward))
        
        return reward


    def reset(self):
        """
        Reset the state of the environment and returns an initial observation.
        Returns
        -------
        observation (object): the initial observation of the space.
        """
        self.curr_episode += 1
        self.ticks = 0
        self.action_episode_memory.append([])
        self.backend.reset_links()
        return self.get_state()

    def render(self, mode='human', close=False):
        return

    def get_state(self):
        """Get the observation.  it is a tuple """
        ob = (self.backend.active_link, self.backend.current_bw1,  self.backend.current_bw2)
        return ob

    def seed(self, seed):
        random.seed(seed)
        np.random.seed


    def cleanup(self):
        self.backend.cleanup()

In [9]:
env = SdwanEnv()
n_actions = env.action_space.n
n_actions

Unable to contact the remote controller at 127.0.0.1:6653


8529.470529470525 10000.0 183.81618381618424 0.0
generating new trffic
Traffic1: 0, Traffic2: 0


/usr/local/lib/python3.9/dist-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


11

In [136]:
import collections
import gym
import numpy as np
import statistics
import tensorflow as tf
import tqdm

from matplotlib import pyplot as plt
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple


# Create the environment


# Set seed for experiment reproducibility
seed = 42
#env.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

# Small epsilon value for stabilizing division operations
eps = np.finfo(np.float32).eps.item()


In [137]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [138]:
class ActorCritic(tf.keras.Model):
  """Combined actor-critic network."""

  def __init__(
      self, 
      num_actions: int, 
      num_hidden_units: int):
    """Initialize."""
    super().__init__()

    self.common = layers.Dense(num_hidden_units, activation="relu")
    self.actor = layers.Dense(num_actions)
    self.critic = layers.Dense(1)

  def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    x = self.common(inputs)
    return self.actor(x), self.critic(x)


In [139]:
num_actions = n_actions  # 2
num_hidden_units = 128

model = ActorCritic(num_actions, num_hidden_units)


In [140]:
def env_step(action: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """Returns state, reward and done flag given an action."""

  state, reward, done, _ = env.step(action)
  d=np.array(state)
  return (d.astype(np.float32), 
          np.array(reward, np.int32), 
          np.array(done, np.int32))


def tf_env_step(action: tf.Tensor) -> List[tf.Tensor]:
  return tf.numpy_function(env_step, [action], 
                           [tf.float32, tf.int32, tf.int32])


In [141]:
def run_episode(
    initial_state: tf.Tensor,  
    model: tf.keras.Model, 
    max_steps: int) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
  """Runs a single episode to collect training data."""

  action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  rewards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)

  initial_state_shape = initial_state.shape
  state = initial_state

  for t in tf.range(max_steps):
    # Convert state into a batched tensor (batch size = 1)
    state = tf.expand_dims(state, 0)

    # Run the model and to get action probabilities and critic value
    action_logits_t, value = model(state)

    # Sample next action from the action probability distribution
    action = tf.random.categorical(action_logits_t, 1)[0, 0]
    action_probs_t = tf.nn.softmax(action_logits_t)

    # Store critic values
    values = values.write(t, tf.squeeze(value))

    # Store log probability of the action chosen
    action_probs = action_probs.write(t, action_probs_t[0, action])

    # Apply action to the environment to get next state and reward
    state, reward, done = tf_env_step(action)
    state.set_shape(initial_state_shape)

    # Store reward
    rewards = rewards.write(t, reward)

    if tf.cast(done, tf.bool):
      break

  action_probs = action_probs.stack()
  values = values.stack()
  rewards = rewards.stack()

  return action_probs, values, rewards

In [142]:
def get_expected_return(
    rewards: tf.Tensor, 
    gamma: float, 
    standardize: bool = True) -> tf.Tensor:
  """Compute expected returns per timestep."""

  n = tf.shape(rewards)[0]
  returns = tf.TensorArray(dtype=tf.float32, size=n)

  # Start from the end of `rewards` and accumulate reward sums
  # into the `returns` array
  rewards = tf.cast(rewards[::-1], dtype=tf.float32)
  discounted_sum = tf.constant(0.0)
  discounted_sum_shape = discounted_sum.shape
  for i in tf.range(n):
    reward = rewards[i]
    discounted_sum = reward + gamma * discounted_sum
    discounted_sum.set_shape(discounted_sum_shape)
    returns = returns.write(i, discounted_sum)
  returns = returns.stack()[::-1]

  if standardize:
    returns = ((returns - tf.math.reduce_mean(returns)) / 
               (tf.math.reduce_std(returns) + eps))

  return returns

In [143]:
huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)

def compute_loss(
    action_probs: tf.Tensor,  
    values: tf.Tensor,  
    returns: tf.Tensor) -> tf.Tensor:
  """Computes the combined actor-critic loss."""

  advantage = returns - values

  action_log_probs = tf.math.log(action_probs)
  actor_loss = -tf.math.reduce_sum(action_log_probs * advantage)

  critic_loss = huber_loss(values, returns)

  return actor_loss + critic_loss

In [144]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)


@tf.function
def train_step(
    initial_state: tf.Tensor, 
    model: tf.keras.Model, 
    optimizer: tf.keras.optimizers.Optimizer, 
    gamma: float, 
    max_steps_per_episode: int) -> tf.Tensor:
  """Runs a model training step."""

  with tf.GradientTape() as tape:

    # Run the model for one episode to collect training data
    action_probs, values, rewards = run_episode(
        initial_state, model, max_steps_per_episode) 

    # Calculate expected returns
    returns = get_expected_return(rewards, gamma)

    # Convert training data to appropriate TF tensor shapes
    action_probs, values, returns = [
        tf.expand_dims(x, 1) for x in [action_probs, values, returns]] 

    # Calculating loss values to update our network
    loss = compute_loss(action_probs, values, returns)

  # Compute the gradients from the loss
  grads = tape.gradient(loss, model.trainable_variables)

  # Apply the gradients to the model's parameters
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  episode_reward = tf.math.reduce_sum(rewards)

  return episode_reward

In [ ]:
min_episodes_criterion = 3000
max_episodes = 10000
max_steps_per_episode = 30

reward_threshold = 30
running_reward = 0

# Discount factor for future rewards
gamma = 0.99

# Keep last episodes reward
episodes_reward: collections.deque = collections.deque(maxlen=min_episodes_criterion)

with tqdm.trange(max_episodes) as t:
    for i in t:
        z=env.reset()
        zz=np.array(z)
        initial_state = tf.constant(zz, dtype=tf.float32)
        episode_reward = int(train_step(
            initial_state, model, optimizer, gamma, max_steps_per_episode))

        episodes_reward.append(episode_reward)
        running_reward = statistics.mean(episodes_reward)

        t.set_description(f'Episode {i}')
        t.set_postfix(
        episode_reward=episode_reward, running_reward=running_reward)

        # Show average episode reward every 10 episodes
        if i % 1 == 0:
            print(f'Episode {i}: average reward: {episode_reward}')

        if i >= min_episodes_criterion:  
            break

print(f'\nSolved at episode {i}: average reward: {running_reward:.2f}!')



  0%|          | 0/10000 [00:00<?, ?it/s]

0 0 0 0


Episode 4:   0%|          | 4/10000 [00:00<31:43,  5.25it/s, episode_reward=172260, running_reward=172260]  

5744.0 5744.0 532.0 532.0
current bw1:5744.0, sla bw:3.0
current bw2:5744.0, sla bw:3.0
Link1 Weight: 9
Link1 Weight: 1
Total Bw: 5744.0, Wanted Bw: 3.0, diff: 5741.0
Chosen action:1, Reward:5742.0
5744.0 5744.0 532.0 532.0
current bw1:5744.0, sla bw:3.0
current bw2:5744.0, sla bw:3.0
Link1 Weight: 9
Link1 Weight: 1
Total Bw: 5744.0, Wanted Bw: 3.0, diff: 5741.0
Chosen action:1, Reward:5742.0
5744.0 5744.0 532.0 532.0
current bw1:5744.0, sla bw:3.0
current bw2:5744.0, sla bw:3.0
Link1 Weight: 9
Link1 Weight: 1
Total Bw: 5744.0, Wanted Bw: 3.0, diff: 5741.0
Chosen action:1, Reward:5742.0
5744.0 5744.0 532.0 532.0
current bw1:5744.0, sla bw:3.0
current bw2:5744.0, sla bw:3.0
Link1 Weight: 9
Link1 Weight: 1
Total Bw: 5744.0, Wanted Bw: 3.0, diff: 5741.0
Chosen action:1, Reward:5742.0
5744.0 5744.0 532.0 532.0
current bw1:5744.0, sla bw:3.0
current bw2:5744.0, sla bw:3.0
Link1 Weight: 9
Link1 Weight: 1
Total Bw: 5744.0, Wanted Bw: 3.0, diff: 5741.0
Chosen action:1, Reward:5742.0
5744.0 574

Episode 8:   0%|          | 7/10000 [00:01<18:20,  9.08it/s, episode_reward=172260, running_reward=172260]

...

Episode 292:   3%|▎         | 293/10000 [00:18<07:04, 22.87it/s, episode_reward=10740, running_reward=1.14e+5]

0 3601.5976035946087 1295.0574138791812 799.8002995506739
current bw1:0, sla bw:3.0
current bw2:3601.5976035946087, sla bw:3.0
Link1 Weight: 9
Link1 Weight: 1
Total Bw: 360.1597603594609, Wanted Bw: 3.0, diff: 357.1597603594609
Chosen action:1, Reward:358.1597603594609
0 3601.5976035946087 1295.0574138791812 799.8002995506739
current bw1:0, sla bw:3.0
current bw2:3601.5976035946087, sla bw:3.0
Link1 Weight: 9
Link1 Weight: 1
Total Bw: 360.1597603594609, Wanted Bw: 3.0, diff: 357.1597603594609
Chosen action:1, Reward:358.1597603594609
0 3601.5976035946087 1295.0574138791812 799.8002995506739
current bw1:0, sla bw:3.0
current bw2:3601.5976035946087, sla bw:3.0
Link1 Weight: 9
Link1 Weight: 1
Total Bw: 360.1597603594609, Wanted Bw: 3.0, diff: 357.1597603594609
Chosen action:1, Reward:358.1597603594609
0 3601.5976035946087 1295.0574138791812 799.8002995506739
current bw1:0, sla bw:3.0
current bw2:3601.5976035946087, sla bw:3.0
Link1 Weight: 9
Link1 Weight: 1
Total Bw: 360.1597603594609, Wa

Episode 297:   3%|▎         | 296/10000 [00:18<07:00, 23.06it/s, episode_reward=10740, running_reward=1.12e+5]

Episode 293: average reward: 10740
0 3601.5976035946087 1295.0574138791812 799.8002995506739
0 3601.5976035946087 1295.0574138791812 799.8002995506739
current bw1:0, sla bw:3.0
current bw2:3601.5976035946087, sla bw:3.0
Link1 Weight: 9
Link1 Weight: 1
Total Bw: 360.1597603594609, Wanted Bw: 3.0, diff: 357.1597603594609
Chosen action:1, Reward:358.1597603594609
0 3601.5976035946087 1295.0574138791812 799.8002995506739
current bw1:0, sla bw:3.0
current bw2:3601.5976035946087, sla bw:3.0
Link1 Weight: 9
Link1 Weight: 1
Total Bw: 360.1597603594609, Wanted Bw: 3.0, diff: 357.1597603594609
Chosen action:1, Reward:358.1597603594609
0 3601.5976035946087 1295.0574138791812 799.8002995506739
current bw1:0, sla bw:3.0
current bw2:3601.5976035946087, sla bw:3.0
Link1 Weight: 9
Link1 Weight: 1
Total Bw: 360.1597603594609, Wanted Bw: 3.0, diff: 357.1597603594609
Chosen action:1, Reward:358.1597603594609
0 3601.5976035946087 1295.0574138791812 799.8002995506739
current bw1:0, sla bw:3.0
current bw2:3

Episode 298:   3%|▎         | 299/10000 [00:18<06:59, 23.14it/s, episode_reward=10740, running_reward=1.12e+5]